<a href="https://colab.research.google.com/github/venugopals002/Venus/blob/main/BOPP_tape_XGboost_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install pandas xgboost gradio

# Import libraries
import pandas as pd
import xgboost as xgb
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00


In [2]:
# Load your data (replace with your CSV file path)
# Upload your CSV to Colab first (use the file uploader on the left panel)
from google.colab import files
uploaded = files.upload()
file_name = next(iter(uploaded))
df = pd.read_csv(file_name)

Saving sales_data.csv to sales_data.csv


In [10]:
# Melt to long format and extract month
df_long = df.melt(id_vars='Item Code', var_name='Month_Year', value_name='Sales')
df_long['Month'] = df_long['Month_Year'].str.extract(r'(January|February|March|April|May|June|July|August|September|October|November|December)')

In [11]:
# Convert month names to numbers
month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}
df_long['Month'] = df_long['Month'].map(month_map)
df_long = df_long.sort_values(['Item Code', 'Month'])

In [12]:
# Feature engineering
df_long['Item_Code_Encoded'] = df_long['Item Code'].astype('category').cat.codes
df_long['Lag_1'] = df_long.groupby('Item Code')['Sales'].shift(1).fillna(0)
df_long['Rolling_Avg_3'] = df_long.groupby('Item Code')['Sales'].transform(lambda x: x.rolling(3, min_periods=1).mean())

In [13]:
# Prepare training data
X = df_long[['Item_Code_Encoded', 'Month', 'Lag_1', 'Rolling_Avg_3']]
y = df_long['Sales']

In [14]:
# Train improved model
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1
)
model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [15]:
# Create prediction function
item_codes = df_long['Item Code'].unique().tolist()

def predict_sales(item_code, month):
    try:
        # Get item history
        item_data = df_long[df_long['Item Code'] == item_code]

        # Prepare features
        item_encoded = pd.Categorical([item_code], categories=item_codes).codes[0]
        last_sales = item_data['Sales'].values[-1] if len(item_data) > 0 else 0
        rolling_avg = item_data['Rolling_Avg_3'].values[-1] if len(item_data) > 0 else 0

        input_data = pd.DataFrame([{
            'Item_Code_Encoded': item_encoded,
            'Month': month,
            'Lag_1': last_sales,
            'Rolling_Avg_3': rolling_avg
        }])

        # Predict and calculate stock
        predicted_sales = max(0, round(model.predict(input_data)[0]))
        safety_stock = round(predicted_sales * 1.5)  # 45-day buffer

        return predicted_sales, safety_stock
    except:
        return 0, 0  # Fallback for new/unseen items

In [16]:
# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## 📈 Smart Inventory Predictor")
    with gr.Row():
        item_input = gr.Dropdown(item_codes, label="Select Item Code")
        month_input = gr.Slider(1, 12, step=1, label="Select Month (1-12)")
    predict_btn = gr.Button("Calculate Inventory Needs")

    with gr.Row():
        sales_output = gr.Number(label="Predicted Sales", precision=0)
        stock_output = gr.Number(label="Recommended Stock", precision=0)

    predict_btn.click(
        fn=predict_sales,
        inputs=[item_input, month_input],
        outputs=[sales_output, stock_output]
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://546ec03d9a76d4fd1d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
